<a href="https://colab.research.google.com/github/devangi2000/Disease-Prediction/blob/master/Sub_section_extraction_BrainTumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import keras
import pandas as pd

# Define a simple one dimensional "image" to extract from
image = np.array([10,11,12,13,14,15])
image

# Compute the dimensions of your "image"
image_length = image.shape[0]
print(image_length)

6


### Sub-sections
we define a "patch size" in three dimensions, that will be the size of the sub-section you want to extract. For this exercise, you only need to define a patch size in one dimension.

In [2]:
# Define a patch length, which will be the size of your extracted sub-section
patch_length = 3

To extract a patch of length `patch_length` you will first define an index at which to start the patch.

Run the next cell to define your start index

In [3]:
# Define your start index
start_i = 0

At the end of the next cell you are adding 1 to the start index. Run cell a few times to extract some one dimensional sub-sections from your "image"

What happens when you run into the edge of the image (when `start_index` is > 3)?

In [4]:
# Define an end index given your start index and patch size
print(f"start index {start_i}")
end_i = start_i + patch_length
print(f"end index {end_i}")

# Extract a sub-section from your "image"
sub_section = image[start_i: end_i]
print("output patch length: ", len(sub_section))
print("output patch array: ", sub_section)

# Add one to your start index
start_i +=1

start index 0
end index 3
output patch length:  3
output patch array:  [10 11 12]


You'll notice when you run the above multiple times, that eventually the sub-section returned is no longer of length `patch_length`. 

your neural network will be expecting a particular sub-section size and will not accept inputs of other dimensions. For the start indices, you will be randomly choosing values and you need to ensure that your random number generator is set up to avoid the edges of your image object.

The next few code cells include a demonstration of how you could determine the constraints on your start index for the simple one dimensional example.

In [5]:
# Set your start index to 3 to extract a valid patch
start_i = 3
print(f"start index {start_i}")
end_i = start_i + patch_length
print(f"end index {end_i}")
sub_section = image[start_i: end_i]
print("output patch array: ", sub_section)

# Compute and print the largest valid value for start index
print(f"The largest start index for which "
      f"a sub section is still valid is "
      f"{image_length - patch_length}")
# Compute and print the range of valid start indices
print(f"The range of valid start indices is:")

# Compute valid start indices, note the range() function excludes the upper bound
valid_start_i = [i for i in range(image_length - patch_length + 1)]
print(valid_start_i)

start index 3
end index 6
output patch array:  [13 14 15]
The largest start index for which a sub section is still valid is 3
The range of valid start indices is:
[0, 1, 2, 3]


### Random selection of start indices
We need to randomly select a valid integer for the start index in each of three dimensions. The way to do this is by following the logic above to identify valid start indices and then selecting randomly from that range of valid numbers.

Run the next cell to select a valid start index for the one dimensional example

In [6]:
# Choose a random start index, note the np.random.randint() function excludes the upper bound.
start_i = np.random.randint(image_length - patch_length + 1)
print(f"randomly selected start index {start_i}")

# Randomly select multiple start indices in a loop
for _ in range(10):
    start_i = np.random.randint(image_length - patch_length + 1)
    print(f"randomly selected start index {start_i}")

randomly selected start index 0
randomly selected start index 3
randomly selected start index 0
randomly selected start index 1
randomly selected start index 3
randomly selected start index 3
randomly selected start index 0
randomly selected start index 3
randomly selected start index 2
randomly selected start index 3
randomly selected start index 0


### Background Ratio

Another thing is to compute the ratio of background to edema and tumorous regions. You will be provided with a file containing labels with these categories:

* 0: background
* 1: edema
* 2: non-enhancing tumor
* 3: enhancing tumor

Let's try to demonstrate this in 1-D to get some intuition on how to implement it in 3D later in the assignment.

In [7]:
# We first simulate input data by defining a random patch of length 16. This will contain labels 
# with the categories (0 to 3) as defined above.

patch_labels = np.random.randint(0, 4, (16))
print(patch_labels)

# A straightforward approach to get the background ratio is
# to count the number of 0's and divide by the patch length

bgrd_ratio = np.count_nonzero(patch_labels == 0) / len(patch_labels)
print("using np.count_nonzero(): ", bgrd_ratio)

bgrd_ratio = len(np.where(patch_labels == 0)[0]) / len(patch_labels)
print("using np.where(): ", bgrd_ratio)

# However, take note that we'll use our label array to train a neural network
# so we can opt to compute the ratio a bit later after we do some preprocessing. 
# First, we convert the label's categories into one-hot format so it can be used to train the model

patch_labels_one_hot = keras.utils.to_categorical(patch_labels, num_classes=4)
print(patch_labels_one_hot)

[3 3 3 0 3 0 0 3 2 1 0 2 2 1 2 3]
using np.count_nonzero():  0.25
using np.where():  0.25
[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


**Note**: We hardcoded the number of classes to 4 in our simple example above.
You should take into account that the label file can have
a different number of categories

In [8]:
# Let's convert the output to a dataframe just so we can see the labels more clearly

pd.DataFrame(patch_labels_one_hot, columns=['background', 'edema', 'non-enhancing tumor', 'enhancing tumor'])


# What we're interested in is the first column because that 
# indicates if the element is part of the background
# In this case, 1 = background, 0 = non-background

print("background column: ", patch_labels_one_hot[:,0])


# we can compute the background ratio by counting the number of 1's 
# in the said column divided by the length of the patch

bgrd_ratio = np.sum(patch_labels_one_hot[:,0])/ len(patch_labels)
print("using one-hot column: ", bgrd_ratio)

background column:  [0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
using one-hot column:  0.25
